IA3配条代码参考自：

[peft_ia3_mindnlp.ipynb](https://github.com/mindspore-lab/mindnlp/tree/master/llm/peft/ia3/seq_2_seq)

模型配置参考：基于MindSpore NLP的Roberta模型Prompt Tuning

https://blog.csdn.net/Kenji_Shinji/article/details/144395136

In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType,IA3Config
from mindnlp.dataset import load_dataset
from mindnlp.core import ops
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm
from mindnlp import evaluate

model_name_or_path = "roberta-large"
tokenizer_name_or_path = "roberta-large"

# checkpoint_name = "financial_sentiment_analysis_lora_v1.ckpt"
checkpoint_name = "RoBERTa_IA3_v1.ckpt"

# 优化训练表现可以调整以下数值
max_length = 128
lr = 1e-4
num_epochs = 1
batch_size = 8

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [2]:
# creating model
peft_config = IA3Config(task_type=TaskType.SEQ_CLS, inference_mode=False) #TaskType从SEQ_2_SEQ_LM修改为SEQ_CLS
#TaskType.SEQ_2_SEQ_LM 是专门为序列到序列生成任务（如机器翻译、文本摘要等）设计的。对于 RoBERTa 模型和 AutoModelForSequenceClassification，你应该将 task_type 设置为 TaskType.SEQ_CLS，因为这是针对序列分类任务的正确配置。
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True) #对应修改为AutoModelForSequenceClassification
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,223,682 || all params: 356,585,476 || trainable%: 0.34316652874555104


In [3]:
#加载tokenizer
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [4]:
# mindspore.dataset.config.set_seed(123)
# loading dataset
# dataset = load_dataset("financial_phrasebank", "sentences_allagree")
datasets = load_dataset("glue", "mrpc")
print(next(datasets['train'].create_dict_iterator()))

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [5]:
from mindnlp.dataset import BaseMapFunction
 
class MapFunc(BaseMapFunction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label
 
def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset
 
train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 67], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,  9064, 32497 ...     1,     1,     1],
 [    0,   133,  4417 ...     1,     1,     1],
 [    0,   133, 19888 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[8, 67], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[8], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1])}


In [7]:
metric = evaluate.load("glue", "mrpc")

In [8]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [9]:
from mindnlp.core import value_and_grad
import mindspore.nn as nn
# training and evaluation

def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

# 定义 Softmax 函数，用于评估阶段的归一化
softmax = nn.Softmax(axis=-1)

# 使用value_and_grad装饰前向传播函数，以便在训练时获取损失值和梯度
grad_fn = value_and_grad(forward_fn, model.trainable_params())

# 开始训练和评估循环
for epoch in range(num_epochs):
    # 设置模型为训练模式
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    # 遍历训练数据集
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        # 清除上一步的梯度
        optimizer.zero_grad()
        # 计算损失值和梯度
        loss = grad_fn(**batch)
         # 更新模型参数
        optimizer.step()
        # 累加损失值
        total_loss += loss.float() #这一步不一样
        # 更新学习率调度器
        lr_scheduler.step()

    # 设置模型为评估模式
    model.set_train(False)
    eval_loss = 0 #这一步没有
    eval_preds = [] #这一步没有
    eval_total_size = eval_dataset.get_dataset_size()
    # 遍历评估数据集
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        # 禁用梯度计算，进行前向传播
        with mindspore._no_grad(): # 没有这句
            outputs = model(**batch)
        loss = outputs.loss        # 以下都不一样
        eval_loss += loss.float()
        # 对于序列分类任务，直接获取类别索引
        print(f'outputs:{outputs}')
        eval_preds.extend(ops.argmax(outputs.logits, -1).asnumpy().tolist())
    
# 计算评估期损失
    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
# 计算训练期损失
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
# 打印训练和评估结果
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  2%|▏         | 1/51 [00:00<00:17,  2.85it/s]

outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.728428), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.48040873e-01,  4.22773451e-01],
 [-4.45561886e-01,  3.94312233e-01],
 [-2.11284250e-01,  7.98318684e-02],
 ...
 [-4.48380202e-01,  4.06560034e-01],
 [-4.30845946e-01,  3.97852480e-01],
 [-4.78388518e-01,  3.75581652e-01]]), hidden_states=None, attentions=None)


  4%|▍         | 2/51 [00:00<00:14,  3.34it/s]

outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.541961), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-3.42127383e-01,  2.15243861e-01],
 [-4.41872180e-01,  4.12963808e-01],
 [-3.49417776e-01,  1.84000209e-01],
 ...
 [-4.23603266e-01,  2.92671263e-01],
 [-4.19992357e-01,  4.01182175e-01],
 [-4.61189926e-01,  4.09017563e-01]]), hidden_states=None, attentions=None)


  6%|▌         | 3/51 [00:00<00:13,  3.68it/s]

...

outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.592777), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-2.11763158e-01,  1.18162133e-01],
 [-4.85430062e-01,  4.47657615e-01],
 [-4.25604403e-01,  3.98084164e-01],
 ...
 [-4.05361950e-01,  3.82383108e-01],
 [-4.30735260e-01,  4.04971957e-01],
 [-4.16345656e-01,  3.95132035e-01]]), hidden_states=None, attentions=None)
outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.698506), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.30772781e-01,  4.00505126e-01],
 [-4.36725408e-01,  4.07539755e-01],
 [-4.15126234e-01,  3.85542214e-01],
 ...
 [-2.65830487e-01,  2.11293519e-01],
 [-4.11877513e-01,  4.00073975e-01],
 [-4.26503062e-01,  3.93677711e-01]]), hidden_states=None, attentions=None)
outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.733483), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.09756750e-01,  3.79970759e-01],
 [-3.96590412e-01

 96%|█████████▌| 49/51 [00:06<00:00, 10.34it/s]

outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.68133), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.33468223e-01,  3.97790104e-01],
 [-5.41779518e-01,  3.92217547e-01],
 [-4.65376824e-01,  4.12408322e-01],
 ...
 [-4.71589714e-01,  4.51715469e-01],
 [-4.01877046e-01,  3.57235223e-01],
 [-4.36540723e-01,  4.10516322e-01]]), hidden_states=None, attentions=None)
outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.589475), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.15849149e-01,  4.02388394e-01],
 [-4.36504543e-01,  3.98892879e-01],
 [-4.13278580e-01,  3.98712993e-01],
 ...
 [-3.83228809e-01,  3.73265237e-01],
 [-4.09423411e-01,  3.98320377e-01],
 [-3.44964504e-01,  2.39860758e-01]]), hidden_states=None, attentions=None)
outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.402563), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-4.35128182e-01,  2.87692398e-01],
 [-2.39549458e-01,

100%|██████████| 51/51 [00:06<00:00,  8.21it/s]

outputs:SequenceClassifierOutput(loss=Tensor(shape=[], dtype=Float32, value= 0.593329), logits=Tensor(shape=[8, 2], dtype=Float32, value=
[[-2.11539447e-01,  1.04477718e-01],
 [-4.36842561e-01,  4.00366277e-01],
 [-4.20866191e-01,  3.93352479e-01],
 ...
 [-3.39094430e-01,  2.59042680e-01],
 [-2.10636526e-01,  1.30466431e-01],
 [-4.56774592e-01,  4.19524759e-01]]), hidden_states=None, attentions=None)
epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.88749) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.635251) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.85164) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.616073)


In [10]:
print(f"{eval_preds=}")

eval_preds=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
# 打印前5个样本的logits
print(f"Sample logits: {outputs.logits[:100]}")

Sample logits: [[-0.21153945  0.10447772]
 [-0.43684256  0.40036628]
 [-0.4208662   0.39335248]
 [-0.4019394   0.31169894]
 [-0.4066868   0.36516586]
 [-0.33909443  0.25904268]
 [-0.21063653  0.13046643]
 [-0.4567746   0.41952476]]


In [28]:
# print accuracy
# 初始化正确预测和总样本的计数器
correct = 0
total = 0

# 初始化用于存储真实标签的列表
ground_truth = []

# 遍历预测结果和验证数据集
for pred, data in zip(eval_preds, datasets['validation'].create_dict_iterator(output_numpy=True)):
    # 获取真实的文本标签
    true = int(data['label'])
    # 将真实标签添加到ground_truth列表中
    ground_truth.append(true)
    # 如果预测的标签与真实标签一致，则正确计数器加一
    if pred == true:
        correct += 1
    # 总样本计数器加一
    total += 1
print(f'correct:{correct},total:{total}')
# 计算准确率
accuracy = correct / total * 100
# 输出准确率
print(f"{accuracy=} % on the evaluation dataset")
# 输出前10个预测结果
print(f"{eval_preds[:10]=}")
# 输出前10个真实标签
print(f"{ground_truth[:10]=}")

correct:279,total:408
accuracy=68.38235294117648 % on the evaluation dataset
eval_preds[:10]=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
ground_truth[:10]=[1, 0, 0, 1, 0, 1, 0, 1, 1, 1]


In [13]:
datasets

{'train': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0xffff8436df40>,
 'validation': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0xfffdfc07d550>,
 'test': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0xfffdfc07d310>}

In [14]:
# 获取数据集迭代器的下一个元素
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 73], dtype=Int64, value=
 [[    0,   894,    26 ...     1,     1,     1],
  [    0, 43600,  1322 ...     1,     1,     1],
  [    0,   133,  1404 ...   135,   479,     2],
  ...
  [    0, 30888,    12 ...     1,     1,     1],
  [    0,  5771,   385 ...     1,     1,     1],
  [    0,   713, 33752 ...     1,     1,     1]]),
 Tensor(shape=[8, 73], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 1, 1, 1],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8], dtype=Int64, value= [1, 0, 0, 1, 0, 1, 0, 1])]

In [15]:
# saving model
# 构建PEFT模型的唯一标识符
# 该标识符结合了模型名称或路径、PEFT配置的类型和任务类型，用于区分不同的PEFT模型
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

# 保存预训练的PEFT模型
# 使用构建的PEFT模型标识符作为模型ID，将模型保存到指定目录或路径
model.save_pretrained(peft_model_id)

In [16]:
# 构建checkpoint文件的路径
ckpt = f"{peft_model_id}/adapter_model.ckpt"
# 使用shell命令检查checkpoint文件的大小
!du -h $ckpt

4.7M	roberta-large_IA3_SEQ_CLS/adapter_model.ckpt


In [17]:
# 导入Peft模型和配置相关的模块
from mindnlp.peft import PeftModel, PeftConfig

# 构造Peft模型的唯一标识符
# 这里使用模型名称或路径、Peft配置的类型和任务类型来组合成一个字符串
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

# 从预训练的Peft模型中加载配置
# 这里的配置将指导如何加载和使用Peft模型
config = PeftConfig.from_pretrained(peft_model_id)
# 从配置中加载基础模型
# 这个基础模型是用于执行序列到序列语言模型任务的
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
# 使用Peft模型对基础模型进行微调
# 这一步是将基础模型与Peft模型的特定微调参数结合起来
model = PeftModel.from_pretrained(model, peft_model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# 设置模型为评估模式，以禁用dropout等训练时的行为
model.set_train(False)
# 从验证数据集中获取一个样本，以便后续进行推理演示
example = next(eval_dataset.create_dict_iterator(output_numpy=True))

# 打印样本'input_ids'、'attention_mask'和'labels'
print(f"Examle:{example}")
# 打印样本中的文本标签，以便用户了解正在处理的数据
print(example['labels'])
# 使用tokenizer对样本文本进行编码，以将其转换为模型可处理的输入格式
inputs = tokenizer(example['labels'], return_tensors="ms")
# 打印编码后的输入，以展示输入格式和内容
print(f"Inputs:{inputs}")

# 禁用梯度计算，以减少计算资源消耗，因为推理过程不需要反向传播
with mindspore._no_grad():
    # 使用模型生成分类结果
    outputs = model(**batch)
    outputs = outputs.logits.argmax(axis=-1)
    # 打印生成的分类结果
    print(f"Outputs:{outputs}")
    
    # 根据MRPC数据集的定义，0代表不相似，1代表相似。将生成的分类结果转换回人类可读的文本格式，并打印
    label_names = ['Dissimilar', 'Similar']
    
    # 获取每个样本的最大概率对应的类别索引
    predicted_class_indices = outputs.asnumpy()
    
    # 将类别索引转换为人类可读的标签
    predicted_labels = [label_names[index] for index in predicted_class_indices]
    
    # 打印分类结果
    for i, label in enumerate(predicted_labels):
        print(f"Sample {i+1}: {label}")

Examle:{'input_ids': array([[    0,   894,    26,     5,   689, 11131, 11637,   265,   630,
          128,    90,  2564,     5,   138,   128,    29,   251,    12,
         1279,   434,  1860,   479,     2,     2,   113,    20,   689,
        11131, 11637,   265,   473,    45,  2564,    84,   251,    12,
         1279,   434,  1860,   479,     2,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1],
       [    0, 43600,  1322, 10054,    26, 14822,   636,  1242, 19975,
            5,  7780,  4304,     8,  1415,   556,     7,   634,    39,
          251,   107,     9,  1058,    11,     5,   997,   479,     2,
            2,  9962,  1141,    26,    37,    21,    22,   727,   135,
          639,  1655,  3516,    22,     8,  1415,   556,     7,   634,
           39,   107,     9,  1058,    1